In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets

pd.set_option('display.max_columns', None)

In [7]:
df = pd.read_csv('Data/indiegogo_kaggle.csv')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20631 entries, 0 to 20630
Data columns (total 74 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   currency            20631 non-null  object 
 1   category            20631 non-null  object 
 2   year_end            20631 non-null  int64  
 3   month_end           20631 non-null  int64  
 4   day_end             20631 non-null  int64  
 5   time_end            20631 non-null  object 
 6   amount_raised       20631 non-null  int64  
 7   funded_percent      20631 non-null  object 
 8   in_demand           20631 non-null  bool   
 9   year_launch         20631 non-null  int64  
 10  month_launch        20631 non-null  int64  
 11  day_launch          20631 non-null  int64  
 12  time_launch         20631 non-null  object 
 13  project_id          20631 non-null  int64  
 14  tagline             20619 non-null  object 
 15  title               20626 non-null  object 
 16  url 

In [15]:
df.head(5)

,currency,category,year_end,month_end,day_end,time_end,amount_raised,funded_percent,in_demand,year_launch,month_launch,day_launch,time_launch,project_id,tagline,title,url,state,date_launch,date_end,amount_raised_usd,goal_usd,australia,canada,switzerland,denmark,western_europe,great_britain,hong_kong,norway,sweden,singapore,united_states,education,productivity,energy_greentech,wellness,comics,fashion_wearables,video_games,photography,tv_shows,dance_theater,phones_accessories,audio,film,transportation,art,environment,writing_publishing,music,travel_outdoors,health_fitness,tabletop_games,home,local_business,food_beverage,culture,human_rights,podcasts_vlogs,camera_gear,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,tperiod
0,USD,Transportation,2010,5,12,23:59:00,840,16.80%,False,2010,4,21,22:38:42,3936,Pure electric motorcycle proves a powerful alt...,Join the Electric Revolution!!!,https://www.indiegogo.com/explore/all?project_...,0,2010-04-21,2010-05-12,840.0,5000.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1,USD,Human Rights,2010,7,2,23:59:00,250,20.83%,False,2010,6,10,17:47:35,5109,Send Me to Haiti...I'm needed there!,Relief Trip to Haiti,https://www.indiegogo.com/explore/all?project_...,0,2010-06-10,2010-07-02,250.0,1200.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2
2,USD,Human Rights,2010,7,10,23:59:00,200,16.67%,False,2010,6,18,09:49:01,5307,Haiti Relief Mission to Leogane,"Out To Reach Leogane, Haiti 2010",https://www.indiegogo.com/explore/all?project_...,0,2010-06-18,2010-07-10,200.0,1200.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,3
3,USD,Photography,2010,10,9,23:59:00,500,25.00%,False,2010,9,9,13:50:48,8731,My third book on Fezziwig Press.,The Transpersonal Papers: 1861-2010,https://www.indiegogo.com/explore/all?project_...,0,2010-09-09,2010-10-09,500.0,2000.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,4
4,USD,Human Rights,2011,1,12,23:59:00,360,0.65%,False,2010,9,14,18:38:51,9165,Homeless Veterans Transitional Housing Develop...,Homeless Veterans need a Hand UP not Hand Out!,https://www.indiegogo.com/explore/all?project_...,0,2010-09-14,2011-01-12,360.0,55000.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,5


In [17]:
df['funded_percent'] = df['funded_percent'].str.replace('%', '').astype(float)
df['is_success'] = (df['funded_percent'] >= 80).astype(int)

/var/folders/mh/t2vz9kkn6z99y7p0rnwcq33m0000gn/T/ipykernel_69517/299455169.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['funded_percent'] = df['funded_percent'].str.replace('%', '').astype(float)
/var/folders/mh/t2vz9kkn6z99y7p0rnwcq33m0000gn/T/ipykernel_69517/299455169.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_success'] = (df['funded_percent'] >= 80).astype(int)


In [24]:
def select_cols_to_remove(df):
    # Create a list of column names
    column_names = df.columns.tolist()

    # Create the select menu
    select_cols = widgets.Select(
        options=column_names,
        rows=5,
        layout=widgets.Layout(width='50%')
    )

    # Create the button to add the selected column to a list
    add_button = widgets.Button(description='Add')
    remove_cols = []

    def add_col_to_list(b):
        col_name = select_cols.value
        if col_name not in remove_cols:
            remove_cols.append(col_name)
        print('Selected columns to remove:', remove_cols)

    add_button.on_click(add_col_to_list)

    # Display the select menu and button
    print("Select columns to remove:")
    display(select_cols)
    display(add_button)

    return remove_cols


In [25]:
# IF NEEDED: Call the function to select columns to remove
cols_to_remove = select_cols_to_remove(df)

Select columns to remove:


Select(layout=Layout(width='50%'), options=('currency', 'category', 'year_end', 'month_end', 'day_end', 'time_…

Button(description='Add', style=ButtonStyle())

In [26]:
df.head(5)

,currency,category,year_end,month_end,day_end,time_end,amount_raised,funded_percent,in_demand,year_launch,month_launch,day_launch,time_launch,project_id,tagline,title,url,state,date_launch,date_end,amount_raised_usd,goal_usd,australia,canada,switzerland,denmark,western_europe,great_britain,hong_kong,norway,sweden,singapore,united_states,education,productivity,energy_greentech,wellness,comics,fashion_wearables,video_games,photography,tv_shows,dance_theater,phones_accessories,audio,film,transportation,art,environment,writing_publishing,music,travel_outdoors,health_fitness,tabletop_games,home,local_business,food_beverage,culture,human_rights,podcasts_vlogs,camera_gear,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,tperiod,is_success
0,USD,Transportation,2010,5,12,23:59:00,840,16.80,False,2010,4,21,22:38:42,3936,Pure electric motorcycle proves a powerful alt...,Join the Electric Revolution!!!,https://www.indiegogo.com/explore/all?project_...,0,2010-04-21,2010-05-12,840.0,5000.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,USD,Human Rights,2010,7,2,23:59:00,250,20.83,False,2010,6,10,17:47:35,5109,Send Me to Haiti...I'm needed there!,Relief Trip to Haiti,https://www.indiegogo.com/explore/all?project_...,0,2010-06-10,2010-07-02,250.0,1200.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0
2,USD,Human Rights,2010,7,10,23:59:00,200,16.67,False,2010,6,18,09:49:01,5307,Haiti Relief Mission to Leogane,"Out To Reach Leogane, Haiti 2010",https://www.indiegogo.com/explore/all?project_...,0,2010-06-18,2010-07-10,200.0,1200.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,3,0
3,USD,Photography,2010,10,9,23:59:00,500,25.00,False,2010,9,9,13:50:48,8731,My third book on Fezziwig Press.,The Transpersonal Papers: 1861-2010,https://www.indiegogo.com/explore/all?project_...,0,2010-09-09,2010-10-09,500.0,2000.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,4,0
4,USD,Human Rights,2011,1,12,23:59:00,360,0.65,False,2010,9,14,18:38:51,9165,Homeless Veterans Transitional Housing Develop...,Homeless Veterans need a Hand UP not Hand Out!,https://www.indiegogo.com/explore/all?project_...,0,2010-09-14,2011-01-12,360.0,55000.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,5,0


In [29]:
df_date_end = df[['year_end', "month_end", "day_end"]]

In [32]:
column_map = {'year_end': 'year', 'month_end': 'month', 'day_end': 'day'}
df_date_end = df_date_end.rename(columns=column_map)

In [33]:
df_date_end['date_end'] = pd.to_datetime(df_date_end, yearfirst=True)

In [35]:
df['end_date'] = df_date_end['date_end']

/var/folders/mh/t2vz9kkn6z99y7p0rnwcq33m0000gn/T/ipykernel_69517/1093333316.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['end_date'] = df_date_end['date_end']


In [37]:
df_date_launch = df[['year_launch', "month_launch", "day_launch"]]
column_map = {'year_launch': 'year', 'month_launch': 'month', 'day_launch': 'day'}
df_date_launch = df_date_launch.rename(columns=column_map)
df_date_launch['date_launch'] = pd.to_datetime(df_date_launch, yearfirst=True)

In [39]:
df['launch_date'] = df_date_launch['date_launch']

/var/folders/mh/t2vz9kkn6z99y7p0rnwcq33m0000gn/T/ipykernel_69517/2145720604.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['launch_date'] = df_date_launch['date_launch']


In [41]:
df['launch_date'] = pd.to_datetime(df['launch_date'])
df['end_date'] = pd.to_datetime(df['end_date'])

In [42]:
df['time_period'] = df.apply(lambda row: pd.Timedelta(row['end_date'] - row['launch_date']).days / 30, axis=1)


In [50]:
df['day_period'] = df.apply(lambda row: pd.Timedelta(row['end_date'] - row['launch_date']).days, axis=1)


In [53]:
df = df.rename(columns={'time_period': 'month_period'})

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20618 entries, 0 to 20630
Data columns (total 79 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   currency            20618 non-null  object        
 1   category            20618 non-null  object        
 2   year_end            20618 non-null  int64         
 3   month_end           20618 non-null  int64         
 4   day_end             20618 non-null  int64         
 5   time_end            20618 non-null  object        
 6   amount_raised       20618 non-null  int64         
 7   funded_percent      20618 non-null  float64       
 8   in_demand           20618 non-null  bool          
 9   year_launch         20618 non-null  int64         
 10  month_launch        20618 non-null  int64         
 11  day_launch          20618 non-null  int64         
 12  time_launch         20618 non-null  object        
 13  project_id          20618 non-null  int64     

In [55]:
df.to_csv('Data/indiepygo_final.csv', index=False)

In [56]:
df['is_success'].value_counts()

0    16033
1     4585
Name: is_success, dtype: int64

In [48]:
indie_df = pd.read_csv('Data/indiepygo_final.csv')

In [49]:
indie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20618 entries, 0 to 20617
Data columns (total 87 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   currency            20618 non-null  object 
 1   category            20618 non-null  object 
 2   year_end            20618 non-null  int64  
 3   month_end           20618 non-null  int64  
 4   day_end             20618 non-null  int64  
 5   time_end            20618 non-null  object 
 6   amount_raised       20618 non-null  int64  
 7   funded_percent      20618 non-null  float64
 8   in_demand           20618 non-null  bool   
 9   year_launch         20618 non-null  int64  
 10  month_launch        20618 non-null  int64  
 11  day_launch          20618 non-null  int64  
 12  time_launch         20618 non-null  object 
 13  project_id          20618 non-null  int64  
 14  tagline             20618 non-null  object 
 15  title               20618 non-null  object 
 16  url 

In [50]:
# Count length of each title
indie_df['title_length'] = indie_df['title'].str.split().str.len()
# Drop name variable
#indie_df.drop('title', axis=1, inplace=True)

In [51]:
indie_df['tagline_length'] = indie_df['tagline'].str.split().str.len()

In [52]:
# Drop name variable
indie_df.drop(['title', 'tagline'], axis=1, inplace=True)

In [53]:
#add day of the week column, based on end date
from datetime import datetime
indie_df['date_end'] = pd.to_datetime(indie_df['date_end'])
day_names = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
indie_df['day_of_week_end'] = indie_df['date_end'].apply(lambda x: day_names[x.weekday()])

In [54]:
#dummifying 
day_dummies = pd.get_dummies(indie_df['day_of_week_end']).add_suffix('_end')
indie_df = pd.concat([indie_df, day_dummies], axis=1)
indie_df.columns = indie_df.columns.str.lower()

In [57]:
indie_df['date_end'] = pd.to_datetime(indie_df['date_end'])
month_names = {1: 'january', 2: 'february', 3: 'march', 4: 'april', 5: 'may', 6: 'june', 7: 'july', 8: 'august', 9: 'september', 10: 'october', 11: 'november', 12: 'december'}
indie_df['month_of_year_end'] = indie_df['date_end'].dt.month.map(month_names)

In [58]:
#dummifying 
month_dummies = pd.get_dummies(indie_df['month_of_year_end']).add_suffix('_end')
indie_df = pd.concat([indie_df, month_dummies], axis=1)
indie_df.columns = indie_df.columns.str.lower()

In [59]:
indie_df.head(10)

,currency,category,year_end,month_end,day_end,time_end,amount_raised,funded_percent,in_demand,year_launch,month_launch,day_launch,time_launch,project_id,url,state,date_launch,date_end,amount_raised_usd,goal_usd,australia,canada,switzerland,denmark,western_europe,great_britain,hong_kong,norway,sweden,singapore,united_states,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,tperiod,is_success,end_date,launch_date,month_period,day_period,art,audio,camera_gear,comics,culture,dance_theater,education,energy_greentech,environment,fashion,film,food_beverages,health_fitness,home,human_rights,local_businesses,music,phones_accessoires,photography,podcasts_blogs,productivity,tabletop_games,transportation,travel_outdoors,video_games,webseries_tvshows,wellness,writing_publishing,day_of_week,friday,monday,saturday,sunday,thursday,tuesday,wednesday,title_length,tagline_length,day_of_week_end,friday_end,monday_end,saturday_end,sunday_end,thursday_end,tuesday_end,wednesday_end,month_of_year_end,april_end,august_end,december_end,february_end,january_end,july_end,june_end,march_end,may_end,november_end,october_end,september_end
0,USD,Transportation,2010,5,12,23:59:00,840,16.80,False,2010,4,21,22:38:42,3936,https://www.indiegogo.com/explore/all?project_...,0,2010-04-21,2010-05-12,840.0,5000.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,2010-05-12,2010-04-21,0.700000,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,Wednesday,0,0,0,0,0,0,1,4,7,Wednesday,0,0,0,0,0,0,1,may,0,0,0,0,0,0,0,0,1,0,0,0
1,USD,Human Rights,2010,7,2,23:59:00,250,20.83,False,2010,6,10,17:47:35,5109,https://www.indiegogo.com/explore/all?project_...,0,2010-06-10,2010-07-02,250.0,1200.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,2,0,2010-07-02,2010-06-10,0.733333,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,Thursday,0,0,0,0,1,0,0,4,6,Friday,1,0,0,0,0,0,0,july,0,0,0,0,0,1,0,0,0,0,0,0
2,USD,Human Rights,2010,7,10,23:59:00,200,16.67,False,2010,6,18,09:49:01,5307,https://www.indiegogo.com/explore/all?project_...,0,2010-06-18,2010-07-10,200.0,1200.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,3,0,2010-07-10,2010-06-18,0.733333,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,Friday,1,0,0,0,0,0,0,6,5,Saturday,0,0,1,0,0,0,0,july,0,0,0,0,0,1,0,0,0,0,0,0
3,USD,Photography,2010,10,9,23:59:00,500,25.00,False,2010,9,9,13:50:48,8731,https://www.indiegogo.com/explore/all?project_...,0,2010-09-09,2010-10-09,500.0,2000.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,4,0,2010-10-09,2010-09-09,1.000000,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,Thursday,0,0,0,0,1,0,0,4,6,Saturday,0,0,1,0,0,0,0,october,0,0,0,0,0,0,0,0,0,0,1,0
4,USD,Human Rights,2011,1,12,23:59:00,360,0.65,False,2010,9,14,18:38:51,9165,https://www.indiegogo.com/explore/all?project_...,0,2010-09-14,2011-01-12,360.0,55000.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,5,0,2011-01-12,2010-09-14,4.000000,120,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,Tuesday,0,0,0,0,0,1,0,9,6,Wednesday,0,0,0,0,0,0,1,january,0,0,0,0,1,0,0,0,0,0,0,0
5,USD,Camera Gear,2011,1,1,23:59:00,1538,69.91,False,2010,11,20,13:26:37,13255,https://www.indiegogo.com/explore/all?project_...,0,2010-11-20,2011-01-01,1538.0,2200.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,6,0,2011-01-01,2010-11-20,1.400000,42,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Saturday,0,0,1,0,0,0,0,5,7,Saturday,0,0,1,0,0,0,0,january,0,0,0,0,1,0,0,0,0,0,0,0
6,USD,Photography,2011,1,21,23:59:00,775,15.50,False,2010,11,22,10:33:00,13333,https://www.indiegogo.com/explore/all?project_...,0,2010-11-22,2011-01-21,775.0,5000.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,7,0,2011-01-21,2010-11-22,2.000000,60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,Monday,0,1,0,0,0,0,0,4,3,Friday,1,0,0,0,0,0,0,january,0,0,0,0,1,0,0,0,0,0,0,0
7,USD,Photography,2011,3,28,23:59:59,71,1.51,False,2010,12,6,10:52:51,14113,https://www.indiegogo.com/explore/all?project_...,0,2010-12-06,2011-03-28,71.0,4700.0,0,0,0,0,0,0,0,0,0,

In [18]:
df_success = df[df['is_success'] == 1]

In [20]:
df_sorted = df_success.sort_values(by='amount_raised_usd', ascending=False)
df_sorted

,Unnamed: 0,currency,category,year_end,month_end,day_end,time_end,amount_raised,funded_percent,in_demand,year_launch,month_launch,day_launch,time_launch,project_id,tagline,title,url,state,date_launch,date_end,amount_raised_usd,goal_usd,australia,canada,switzerland,denmark,western_europe,great_britain,hong_kong,norway,sweden,singapore,united_states,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec,tperiod,is_success,end_date,launch_date,month_period,day_period,art,audio,camera_gear,comics,culture,dance_theater,education,energy_greentech,environment,fashion,film,food_beverages,health_fitness,home,human_rights,local_businesses,music,phones_accessoires,photography,podcasts_blogs,productivity,tabletop_games,transportation,travel_outdoors,video_games,webseries_tvshows,wellness,writing_publishing,day_of_week,friday,monday,saturday,sunday,thursday,tuesday,wednesday
13798,13798,USD,Transportation,2018,9,28,23:59:59,17595741,23655.49,False,2018,7,16,22:12:28,2372181,"The coolest, most powerful, high spec'd eBike ...",MATE X: The coolest foldable eBike EVER,https://www.indiegogo.com/explore/all?project_...,1,2018-07-16,2018-09-28,1.759574e+07,74383.3229,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,14515,1,2018-09-28,2018-07-16,2.466667,74,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,Monday,0,1,0,0,0,0,0
12649,12649,USD,Energy & Green Tech,2018,3,4,23:59:59,14959087,18983.50,False,2018,3,3,23:59:59,2332464,Honey on tap directly from your beehive,Flow Hive 2,https://www.indiegogo.com/explore/all?project_...,1,2018-03-03,2018-03-04,1.495909e+07,78800.4748,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,13245,1,2018-03-04,2018-03-03,0.033333,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Saturday,0,0,1,0,0,0,0
18833,18833,USD,Transportation,2020,6,7,23:59:59,13174786,60526.10,True,2020,3,17,00:00:00,2587745,Leave boring behind. Classic look with modern ...,The Babymaker - Stealth Road eBike With Belt D...,https://www.indiegogo.com/explore/all?project_...,1,2020-03-17,2020-06-07,1.317479e+07,21767.1171,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,20099,1,2020-06-07,2020-03-17,2.733333,82,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,Tuesday,0,0,0,0,0,1,0
18843,18843,USD,Transportation,2020,6,7,23:59:59,13173068,60526.10,True,2020,3,17,00:00:00,2587745,Leave boring behind. Classic look with modern ...,The Babymaker - Stealth Road eBike With Belt D...,https://www.indiegogo.com/explore/all?project_...,1,2020-03-17,2020-06-07,1.317307e+07,21764.2787,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,20109,1,2020-06-07,2020-03-17,2.733333,82,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,Tuesday,0,0,0,0,0,1,0
16913,16913,EUR,Productivity,2019,10,14,23:59:59,6854016,10420.46,True,2019,8,20,00:00:00,2536338,The smart & fully wireless native Full HD proj...,Philips PicoPix Max - 1080p Full HD Pico Proje...,https://www.indiegogo.com/explore/all?project_...,1,2019-08-20,2019-10-14,7.775533e+06,74617.9328,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,18056,1,2019-10-14,2019-08-20,1.833333,55,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,Tuesday,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,806,USD,Photography,2014,7,8,23:59:59,400,80.00,False,2014,6,23,17:08:36,835782,Fundraising for my debut photography exhibiti...,Debut Photography Exhibition Fundraiser,https://www.indiegogo.com/explore/all?project_...,0,2014-06-23,2014-07-08,4.000000e+02,500.0000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,827,1,2014-07-08,2014-06-23,0.500000,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,Monday,0,1,0,0,0,0,0
2179,2179,USD,Health & Fitness,2015,4,8,23:59:59,400,80.00,False,2015,3,18,13:29

In [ ]:
#create target value column, based on funded_percent
indie_df['funded_percent'] = indie_df['funded_percent'].str.replace('%', '').astype(float)
indie_df['is_success'] = (indie_df['funded_percent'] >= 80).astype(int)

In [4]:
#add day of the week column, based on launch date
from datetime import datetime
indie_df['date_launch'] = pd.to_datetime(indie_df['date_launch'])
day_names = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
indie_df['day_of_week_end'] = indie_df['date_launch'].apply(lambda x: day_names[x.weekday()])

In [35]:
#dummifying 
day_dummies = pd.get_dummies(indie_df['day_of_week']).add_suffix('_end')
indie_df = pd.concat([indie_df, day_dummies], axis=1)
indie_df.columns = indie_df.columns.str.lower()

In [45]:
indie_df['date_launch'] = pd.to_datetime(indie_df['date_launch'])
month_names = {1: 'january', 2: 'february', 3: 'march', 4: 'april', 5: 'may', 6: 'june', 7: 'july', 8: 'august', 9: 'september', 10: 'october', 11: 'november', 12: 'december'}
indie_df['month_of_year'] = indie_df['date_launch'].dt.month.map(month_names)

In [46]:
#dummifying 
month_dummies = pd.get_dummies(indie_df['month_of_year']).add_suffix('_end')
indie_df = pd.concat([indie_df, month_dummies], axis=1)
indie_df.columns = indie_df.columns.str.lower()

In [47]:
indie_df = indie_df.dropna(how='any')
indie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20618 entries, 0 to 20617
Columns: 103 entries, currency to wednesday_end
dtypes: bool(1), datetime64[ns](2), float64(4), int64(72), object(10), uint8(14)
memory usage: 14.1+ MB


In [60]:
indie_df.to_csv('Data/indiepygo_final.csv', index=False) #save pdf

In [11]:
df = pd.read_csv('Data/indiepygo_final.csv')

In [13]:
df.drop(['Unnamed: 0'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20618 entries, 0 to 20617
Data columns (total 88 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          20618 non-null  int64  
 1   currency            20618 non-null  object 
 2   category            20618 non-null  object 
 3   year_end            20618 non-null  int64  
 4   month_end           20618 non-null  int64  
 5   day_end             20618 non-null  int64  
 6   time_end            20618 non-null  object 
 7   amount_raised       20618 non-null  int64  
 8   funded_percent      20618 non-null  float64
 9   in_demand           20618 non-null  bool   
 10  year_launch         20618 non-null  int64  
 11  month_launch        20618 non-null  int64  
 12  day_launch          20618 non-null  int64  
 13  time_launch         20618 non-null  object 
 14  project_id          20618 non-null  int64  
 15  tagline             20618 non-null  object 
 16  titl

In [21]:
indie_text_df = df[['title', 'tagline', 'is_success']]
indie_text_df

,title,tagline,is_success
0,Join the Electric Revolution!!!,Pure electric motorcycle proves a powerful alt...,0
1,Relief Trip to Haiti,Send Me to Haiti...I'm needed there!,0
2,"Out To Reach Leogane, Haiti 2010",Haiti Relief Mission to Leogane,0
3,The Transpersonal Papers: 1861-2010,My third book on Fezziwig Press.,0
4,Homeless Veterans need a Hand UP not Hand Out!,Homeless Veterans Transitional Housing Develop...,0
...,...,...,...
20613,Totally Gay Productions,Trans filmmaker making queer documentaries. It...,0
20614,Lady Crow debut EP,"Help us finance Every Stone, our first EP!",0
20615,"JOSA, the ultimate Venice-Inspired bracelet","Benefits Save Venice, helps to protect Venice'...",0
20616,My Campaign Title,Hi,0


In [22]:
indie_text_df['text'] = indie_text_df['title'] + ' ' + indie_text_df['tagline']
indie_text_df['text']

/var/folders/mh/t2vz9kkn6z99y7p0rnwcq33m0000gn/T/ipykernel_64180/2641864021.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indie_text_df['text'] = indie_text_df['title'] + ' ' + indie_text_df['tagline']


0        Join the Electric Revolution!!! Pure electric ...
1        Relief Trip to Haiti Send Me to Haiti...I'm ne...
2        Out To Reach Leogane, Haiti 2010 Haiti Relief ...
3        The Transpersonal Papers: 1861-2010 My third b...
4        Homeless Veterans need a Hand UP not Hand Out!...
                               ...                        
20613    Totally Gay Productions Trans filmmaker making...
20614    Lady Crow debut EP Help us finance Every Stone...
20615    JOSA, the ultimate Venice-Inspired bracelet Be...
20616                                 My Campaign Title Hi
20617                                 My Campaign Title Hi
Name: text, Length: 20618, dtype: object

In [23]:
indie_text_df.drop(['title', 'tagline'], axis=1, inplace=True)


/var/folders/mh/t2vz9kkn6z99y7p0rnwcq33m0000gn/T/ipykernel_64180/2395777452.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indie_text_df.drop(['title', 'tagline'], axis=1, inplace=True)


In [24]:
indie_text_df

,is_success,text
0,0,Join the Electric Revolution!!! Pure electric ...
1,0,Relief Trip to Haiti Send Me to Haiti...I'm ne...
2,0,"Out To Reach Leogane, Haiti 2010 Haiti Relief ..."
3,0,The Transpersonal Papers: 1861-2010 My third b...
4,0,Homeless Veterans need a Hand UP not Hand Out!...
...,...,...
20613,0,Totally Gay Productions Trans filmmaker making...
20614,0,Lady Crow debut EP Help us finance Every Stone...
20615,0,"JOSA, the ultimate Venice-Inspired bracelet Be..."
20616,0,My Campaign Title Hi


In [18]:
indie_text_df.to_csv('indiepygo_text_fornlp.csv', index=False)

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    indie_text_df.text, 
    indie_text_df.is_success, 
    test_size=0.2, 
    random_state=2022,
    stratify=indie_text_df.is_success
)

In [22]:
y_train.value_counts()

0    12826
1     3668
Name: is_success, dtype: int64

In [23]:
y_test.value_counts()

0    3207
1     917
Name: is_success, dtype: int64

In [3]:
from sklearn.pipeline import Pipeline

In [26]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('Random Forest', RandomForestClassifier())         
])


clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)



print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.98      0.88      3207
           1       0.66      0.12      0.21       917

    accuracy                           0.79      4124
   macro avg       0.73      0.55      0.54      4124
weighted avg       0.77      0.79      0.73      4124



In [27]:
from sklearn.naive_bayes import MultinomialNB


clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('Multi NB', MultinomialNB())         
])


clf.fit(X_train, y_train)



y_pred = clf.predict(X_test)



print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.99      0.88      3207
           1       0.76      0.06      0.11       917

    accuracy                           0.79      4124
   macro avg       0.77      0.53      0.50      4124
weighted avg       0.78      0.79      0.71      4124



In [28]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('KNN', KNeighborsClassifier())         
])


clf.fit(X_train, y_train)



y_pred = clf.predict(X_test)



print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      1.00      0.88      3207
           1       0.79      0.03      0.06       917

    accuracy                           0.78      4124
   macro avg       0.79      0.51      0.47      4124
weighted avg       0.79      0.78      0.69      4124



/Users/juliettedegoul/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [9]:
indie_text_df['text'] = indie_text_df['text'].str.lower()
indie_text_df['text'] = indie_text_df['text'].str.replace('[^\w\s]','')

/var/folders/mh/t2vz9kkn6z99y7p0rnwcq33m0000gn/T/ipykernel_64180/2820257893.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indie_text_df['text'] = indie_text_df['text'].str.lower()
/var/folders/mh/t2vz9kkn6z99y7p0rnwcq33m0000gn/T/ipykernel_64180/2820257893.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  indie_text_df['text'] = indie_text_df['text'].str.replace('[^\w\s]','')
/var/folders/mh/t2vz9kkn6z99y7p0rnwcq33m0000gn/T/ipykernel_64180/2820257893.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [10]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
indie_text_df['text'] = indie_text_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/juliettedegoul/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/var/folders/mh/t2vz9kkn6z99y7p0rnwcq33m0000gn/T/ipykernel_64180/2816505140.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indie_text_df['text'] = indie_text_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    indie_text_df.text, 
    indie_text_df.is_success, 
    test_size=0.2, 
    random_state=2022,
    stratify=indie_text_df.is_success
)

In [33]:
clf = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('Random Forest', RandomForestClassifier())         
])


clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)



print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.98      0.88      3207
           1       0.66      0.16      0.26       917

    accuracy                           0.79      4124
   macro avg       0.73      0.57      0.57      4124
weighted avg       0.77      0.79      0.74      4124



In [15]:
from gensim.models import Word2Vec
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

clf = Pipeline([
     ('Word2Vec',Word2Vec(sentences=indie_text_df['text'], vector_size=100, window=5, min_count=1, workers=4)),    
     ('Random Forest', RandomForestClassifier())         
])


clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)



print(classification_report(y_test, y_pred))


TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'Word2Vec(vocab=425, vector_size=100, alpha=0.025)' (type <class 'gensim.models.word2vec.Word2Vec'>) doesn't

In [28]:
def get_sentence_embedding(w2v_model, sentence):
    """
    Compute the sentence embedding using the average of word embeddings.
    """
    words = sentence.split()
    words_vectors = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vectors) == 0:
        return np.zeros(w2v_model.vector_size)
    else:
        return np.mean(words_vectors, axis=0)

In [29]:
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import pandas as pd


# Train/test split
X_train, X_test, y_train, y_test = train_test_split(indie_text_df['text'], indie_text_df['is_success'], test_size=0.2, random_state=42, stratify=indie_text_df['is_success'])

# Train Word2Vec
w2v_model = Word2Vec(sentences=X_train, vector_size=100, window=5, min_count=1, workers=4)

# Transform the text data to embeddings
X_train_embeddings = [get_sentence_embedding(w2v_model, sentence) for sentence in X_train]
X_test_embeddings = [get_sentence_embedding(w2v_model, sentence) for sentence in X_test]

# Fit the random forest model
rf = RandomForestClassifier()
rf.fit(X_train_embeddings, y_train)

# Predict on the test set
y_pred = rf.predict(X_test_embeddings)

# Print the classification report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.78      0.99      0.87      3207
           1       0.48      0.04      0.07       917

    accuracy                           0.78      4124
   macro avg       0.63      0.51      0.47      4124
weighted avg       0.72      0.78      0.70      4124

